In [1]:
import tensorflow as tf
import keras.backend as K
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from Project.dataStructures import *
from Project.manageDataset import *
from Project.dataStructures import *
from Project.dataStructures import get_data_target_lists
from Project.launchAlgorithms import *
from Project.utilities import *
from keras.models import Sequential
from keras.layers import PReLU, Dense, Activation,BatchNormalization,Dropout
import numpy as np

Using TensorFlow backend.
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
K.set_learning_phase(0)


In [3]:
rdm = np.random.RandomState()
rdm.seed(42)

In [4]:
datasetList, attributes, coloumns = read_file('/dataset.csv')
targetsDic, targets = get_classes(coloumns['class'])
dataset = string_to_float(switch_label(datasetList, targets))
rdm.shuffle(dataset)
labels = []
for item in dataset:
    labels.append(item[-1])  
stringLabels = [targets[item-1] for item in labels]




In [5]:
X = np.array(dataset)[:,:len(dataset[0])-1].astype(np.float32)
Y = np.array(dataset)[:,len(dataset[0])-1].astype(np.int32)
Y = np.array([item-1 for item in Y])
xTrain, xTest = X[:int(999*0.8)], X[int(999*0.8):]
yTrain, yTest = Y[:int(999*0.8)], Y[int(999*0.8):]


In [6]:
scaler = MinMaxScaler(feature_range=(0,1))
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

In [7]:
layers = [
    Dense(50, input_dim=len(xTrain[0]),init='he_normal'), PReLU(), BatchNormalization(), Dropout(0.2),
    Dense(10, init='he_uniform'), Activation('softmax')
]

In [8]:
model = Sequential()
for layer in layers:
    model.add(layer)


In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta')

In [10]:
for e in range(100):
    losses = []
    for x_batch,y_batch in get_minibatches(xTrain,yTrain):
        loss = model.train_on_batch(x_batch,y_batch)
        losses.append(loss)
    
    train_acc = 0.
    for x_batch,y_batch in get_minibatches(xTrain,yTrain):
        train_acc += (np.argmax(model.predict(x_batch), axis=1) == y_batch[:,0]).sum()
    test_acc = 0.
    for x_batch,y_batch in get_minibatches(xTest,yTest):
        test_acc += (np.argmax(model.predict(x_batch), axis=1) == y_batch[:,0]).sum()
        
    print 'Loss: %.3f - Train Acc: %.3f - Test Acc: %.3f' % (np.mean(losses), 100.*train_acc/len(yTrain),100.*test_acc/len(yTest))

Loss: 2.293 - Train Acc: 13.892 - Test Acc: 17.910
Loss: 2.213 - Train Acc: 24.155 - Test Acc: 21.891
Loss: 2.091 - Train Acc: 31.289 - Test Acc: 34.826
Loss: 1.923 - Train Acc: 39.800 - Test Acc: 39.801
Loss: 1.745 - Train Acc: 44.806 - Test Acc: 42.786
Loss: 1.585 - Train Acc: 47.309 - Test Acc: 45.274
Loss: 1.469 - Train Acc: 50.814 - Test Acc: 50.249
Loss: 1.371 - Train Acc: 53.942 - Test Acc: 47.761
Loss: 1.300 - Train Acc: 60.826 - Test Acc: 56.716
Loss: 1.243 - Train Acc: 61.577 - Test Acc: 57.711
Loss: 1.192 - Train Acc: 64.581 - Test Acc: 59.701
Loss: 1.143 - Train Acc: 64.205 - Test Acc: 56.219
Loss: 1.115 - Train Acc: 65.707 - Test Acc: 61.194
Loss: 1.087 - Train Acc: 66.708 - Test Acc: 61.692
Loss: 1.066 - Train Acc: 67.835 - Test Acc: 63.682
Loss: 1.022 - Train Acc: 66.583 - Test Acc: 59.204
Loss: 1.018 - Train Acc: 68.836 - Test Acc: 63.184
Loss: 0.989 - Train Acc: 67.835 - Test Acc: 61.194
Loss: 0.978 - Train Acc: 70.213 - Test Acc: 64.179
Loss: 0.969 - Train Acc: 68.335

KeyboardInterrupt: 

In [11]:
layers = [
    Dense(50, input_dim=len(xTrain[0]),init='he_normal'), PReLU(), Dropout(0.2), #BatchNormalization(), 
    Dense(20, init='he_normal'), PReLU(), Dropout(0.2),
    Dense(50, init='he_normal'), PReLU(),
    Dense(len(xTrain[0]), 'he_uniform'),Activation('sigmoid')
]

In [12]:
ae = Sequential()
for layer in layers:
    ae.add(layer)

In [13]:
ae.compile(loss='binary_crossentropy', optimizer='adadelta')

In [14]:
for e in range(100):
    losses = []
    for xBatch,yBatch in get_minibatches(X,Y):
        loss = ae.train_on_batch(xBatch,xBatch)
        losses.append(loss)
    print loss

-5.52384
-6.28715
-6.24408
-6.54344
-6.16925
-6.26607
-6.31527
-6.30145
-6.26018
-6.39927
-6.23213
-6.2426
-6.38079
-6.49266
-6.53403
-6.46514
-6.22063
-6.46159
-6.28227
-6.28741
-6.30933
-6.30944
-6.26362
-6.35077
-6.32898
-6.51234
-6.41488
-6.44393
-6.32946
-6.1787
-6.40876
-6.21417
-6.33956
-6.59255
-6.16513
-6.38882
-6.293
-6.47128
-6.01663
-6.46239
-6.23547
-6.31212
-6.39921
-6.36226
-6.24814
-6.38934
-6.22799
-6.50863
-6.47622
-6.48242
-6.44334
-6.33062
-6.48736
-6.43973
-6.47357
-6.54657
-6.27299
-6.21739
-6.44638
-6.30953
-6.25796
-6.26687
-6.45796
-6.44663
-6.10326
-6.23155
-6.32096
-6.37618
-6.32622
-6.31378
-6.27142
-6.26681
-6.31353
-6.27322
-6.33339
-6.27624
-6.58923
-6.39704
-6.38665
-6.44043
-6.32917
-6.48509
-6.27438
-6.24405
-6.66911
-6.39949
-6.44561
-6.59397
-6.28255
-6.36048
-6.38685
-6.48742
-6.38326
-6.20546
-6.60848
-6.42267
-6.2899
-6.35908
-6.53971
-6.52452


In [15]:
embedding = K.function([ae.layers[0].input, 0], [ae.layers[3].output])

In [22]:
newX = []
stringLabels = [targets[item-1] for item in labels]
outputX = embedding([X])[0]
for item in outputX:
    newX.append(item)

dataset = np.c_[np.array(newX), np.array(stringLabels)].tolist() 
    

In [17]:
with open('encodedDataset' + '.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(dataset)
